## Parse cert data
TY to pyca for doing all the heavy lifting.

See https://cryptography.io/en/latest/x509/reference/#cryptography.x509.Certificate and https://cryptography.io/en/latest/x509/reference/#cryptography.x509.Certificate.

Pivoted away from PyCryptodome API which parses [PEM](https://github.com/Legrandin/pycryptodome/blob/fc272f6c0d85779cff67c928c798ef68f532923e/lib/Crypto/IO/PEM.py) but does not appear to do much for the X.509 format.

In [1]:
from cryptography import x509

In [2]:
with open("google-com.pem", "rb") as pem_file:
    cert = x509.load_pem_x509_certificate(pem_file.read())

## Check signature
https://www.pycryptodome.org/src/signature/dsa

In [3]:
from Crypto.Signature import DSS
from Crypto.PublicKey import ECC

Google Trust Services WE2 is the issuer that signed the `google.com` cert.

In [4]:
pubkey = "04:35:7E:1F:F2:14:ED:90:7D:E1:9E:2A:34:43:86:C1:D5:96:E8:27:70:DF:9E:04:CB:A9:CA:86:79:0B:08:4D:46:8A:C2:74:A4:BB:D9:BF:EE:FD:23:D7:38:F3:4B:EF:54:17:E1:BE:E7:CA:55:25:A8:0C:30:AC:2D:5D:4E:A1:51"
len(pubkey)

194

https://www.pycryptodome.org/src/public_key/ecc#Crypto.PublicKey.ECC.import_key

In [5]:
key = ECC.import_key(bytes.fromhex(pubkey.replace(":", "")), curve_name="p256")
key

EccKey(curve='NIST P-256', point_x=24195424185723013134886565302946171634531305375505897135578359356117641350470, point_y=62762746478848224863140673215634629263664737480104886098215293662380625404241)

In [6]:
from Crypto.Hash import SHA256

See https://cryptography.io/en/latest/x509/reference/#cryptography.x509.Certificate.tbs_certificate_bytes.

In [7]:
h = SHA256.new(cert.tbs_certificate_bytes)
h.hexdigest()

'd38564210b319bbcf48ed85865f88bf780cbca4794be2b8e7dccd94914cd4155'

Copy & paste signature from `openssl x509` output.

In [8]:
sig_value = "30:45:02:21:00:ac:6f:77:9b:5e:3e:8d:14:93:ce:95:4f:fa:cf:61:80:8a:2c:0a:b0:3d:36:d8:f8:17:74:52:db:67:f9:08:ad:02:20:58:9d:dc:2e:a1:f5:fe:35:b8:00:c9:cc:91:ec:c9:3e:80:12:57:05:a5:37:16:8f:6a:39:9c:ea:96:09:0e:94"

In [9]:
sig = bytes.fromhex(sig_value.replace(":", ""))
len(sig)

71

In [10]:
assert sig == cert.signature # Sanity check.

In [11]:
verifier = DSS.new(key, mode="fips-186-3", encoding="der")

In [12]:
try:
    verifier.verify(h, sig)
    print("Valid signature.")
except ValueError:
    print("invalid signature.")


Valid signature.
